In [2]:
import sqlite3

# Create a connection to the SQLite database
conn = sqlite3.connect("superhero_db.sqlite")

# Create a cursor object to execute SQL commands
cursor = conn.cursor()


In [3]:
# Creating tables
cursor.executescript('''
CREATE TABLE IF NOT EXISTS heroes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    power TEXT NOT NULL,
    team TEXT
);

CREATE TABLE IF NOT EXISTS villains (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    evil_plan TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS battles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hero_id INTEGER NOT NULL,
    villain_id INTEGER NOT NULL,
    outcome TEXT NOT NULL CHECK (outcome IN ('Win', 'Loss', 'Draw')),
    FOREIGN KEY (hero_id) REFERENCES heroes(id),
    FOREIGN KEY (villain_id) REFERENCES villains(id)
);
''')

# Commit and close connection
conn.commit()
conn.close()


In [4]:
import sqlite3
import pandas as pd

heroes_df = pd.read_csv("heroes.csv")
villains_df = pd.read_csv("villains.csv")
battles_df = pd.read_csv("battles.csv")

In [6]:
# Connect to the database
conn = sqlite3.connect("superhero_db.sqlite")
cursor = conn.cursor()

# Insert heroes data
heroes_df.to_sql("heroes", conn, if_exists="replace", index=False)

# Insert villains data
villains_df.to_sql("villains", conn, if_exists="replace", index=False)

# Insert battles data
battles_df.to_sql("battles", conn, if_exists="replace", index=False)

# Commit changes and close connection
conn.commit()
conn.close()


In [12]:
conn = sqlite3.connect("superhero_db.sqlite")

query_heroes = "SELECT name, power FROM heroes"
heroes_data = pd.read_sql(query_heroes, conn)

print(heroes_data)

query_won_battles = """
SELECT h.name AS hero_name, v.name AS villain_name, b.outcome
FROM battles b
JOIN heroes h ON b.hero_id = h.id
JOIN villains v ON b.villain_id = v.id
WHERE b.outcome = 'Hero Wins'
"""
won_battles_data = pd.read_sql(query_won_battles, conn)

print(won_battles_data)

query_villains = "SELECT name, evil_plan FROM villains"
villains_data = pd.read_sql(query_villains, conn)

print(villains_data)

query_most_battles = """
SELECT h.name AS hero_name, COUNT(b.id) AS battle_count
FROM battles b
JOIN heroes h ON b.hero_id = h.id
GROUP BY h.name
ORDER BY battle_count DESC
LIMIT 1
"""
most_battles_data = pd.read_sql(query_most_battles, conn)

print(most_battles_data)

conn.close()

             name               power
0      Spider-Man        Web-slinging
1        Iron Man       Powered Armor
2          Batman    Genius Detective
3        Superman      Super Strength
4            Thor      God of Thunder
5    Wonder Woman  Amazonian Strength
6   Black Panther    Enhanced Agility
7       The Flash         Super Speed
8            Hulk      Super Strength
9  Doctor Strange         Mystic Arts
       hero_name villain_name    outcome
0           Hulk        Joker  Hero Wins
1     Spider-Man     Darkseid  Hero Wins
2           Hulk        Venom  Hero Wins
3         Batman       Thanos  Hero Wins
4  Black Panther         Loki  Hero Wins
5      The Flash      Magneto  Hero Wins
6         Batman   Lex Luthor  Hero Wins
7         Batman         Loki  Hero Wins
8           Thor         Loki  Hero Wins
           name                           evil_plan
0  Green Goblin                  Terrorize New York
1        Thanos         Collect all Infinity Stones
2         Joker 

In [13]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("superhero_db.sqlite")
cursor = conn.cursor()

# Create a new table for villain hideouts
cursor.execute('''
CREATE TABLE IF NOT EXISTS villain_hideouts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    villain_id INTEGER NOT NULL,
    hideout TEXT NOT NULL,
    FOREIGN KEY (villain_id) REFERENCES villains(id)
);
''')
conn.commit()
conn.close()


In [14]:
conn = sqlite3.connect("superhero_db.sqlite")
cursor = conn.cursor()

# Insert sample hideouts for villains
hideout_data = [
    (1, "Green Goblin's Lab"),
    (2, "Lex Luthor's Tower"),
    (3, "The Joker's Funhouse"),
    (4, "Doctor Doom's Castle"),
    (5, "Thanos' Sanctuary"),
    (6, "Red Skull's Hydra Base"),
    (7, "Darkseid's Apokolips"),
    (8, "Galactus' Asteroid"),
    (9, "Magneto's Fortress"),
    (10, "Ra's al Ghul's Lair"),
]

# Insert data into the table
cursor.executemany("INSERT INTO villain_hideouts (villain_id, hideout) VALUES (?, ?)", hideout_data)

# Commit and close connection
conn.commit()
conn.close()

In [15]:
conn = sqlite3.connect("superhero_db.sqlite")

# Query to retrieve villains and their hideouts
query_hideouts = """
SELECT v.name AS villain_name, h.hideout
FROM villain_hideouts h
JOIN villains v ON h.villain_id = v.id
"""

villain_hideouts_data = pd.read_sql(query_hideouts, conn)

print(villain_hideouts_data)

conn.close()

   villain_name                 hideout
0  Green Goblin      Green Goblin's Lab
1        Thanos      Lex Luthor's Tower
2         Joker    The Joker's Funhouse
3    Lex Luthor    Doctor Doom's Castle
4          Loki       Thanos' Sanctuary
5        Ultron  Red Skull's Hydra Base
6      Darkseid    Darkseid's Apokolips
7         Venom      Galactus' Asteroid
8       Magneto      Magneto's Fortress
9     Red Skull     Ra's al Ghul's Lair
